In [1]:
import pandas as pd
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score,
    precision_recall_fscore_support
)
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)



In [2]:
data = pd.read_excel('BD_numericas.xlsx')


In [3]:
df = data


In [4]:
df.head()

,id,NIVEL ESTUDIOS,ESTADOCIVIL,GÉNERO,TIPO VIVIENDA,TIEMPO EN RESIDENCIA,ESTRATO,TIPO CONTRATO,TIEMPO ACTIVIDAD,INGRESOS FIJOS,...,CUOTAMENSUAL,CODIGO CIIU,SCORE,SALDO CAPITAL,SALDO VENCIDO,DIAS DE MORA,FRANJA DE MORA,EDAD,DIAS EN APROBAR,TARGET
0,0,5,0,0.0,2,3.0,5,0,5.0,13000000,...,632001.190699,8560,649,0,0.0,0,0,42,0,1
1,1,3,0,1.0,1,21.0,4,1,21.0,19000000,...,232260.179192,4610,843,5141500,0.0,0,0,60,0,1
2,2,3,3,1.0,2,15.0,3,0,1.0,1500000,...,91504.941888,4799,630,1259000,0.0,0,0,20,0,1
3,3,3,3,1.0,1,7.0,4,0,4.0,4000000,...,115102.274644,9609,754,2492576,0.0,0,0,29,0,1
4,4,5,0,1.0,1,19.0,4,2,1.0,16000000,...,84800.457032,4799,503,1840083,0.0,0,0,18,0,1


In [5]:
y = (df["DIAS DE MORA"] >= 30).astype(int)
print("0=<30, 1=≥30")
print(y.value_counts(normalize=True).rename("proporción").round(3))


0=<30, 1=≥30
DIAS DE MORA
0    0.842
1    0.158
Name: proporción, dtype: float64


In [6]:
X = df.drop(columns=["DIAS DE MORA", "FRANJA DE MORA", "TARGET"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45816 entries, 0 to 45815
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        45816 non-null  int64  
 1   NIVEL ESTUDIOS            45816 non-null  int64  
 2   ESTADOCIVIL               45816 non-null  int64  
 3   GÉNERO                    44628 non-null  float64
 4   TIPO VIVIENDA             45816 non-null  int64  
 5   TIEMPO EN RESIDENCIA      45816 non-null  float64
 6   ESTRATO                   45816 non-null  int64  
 7   TIPO CONTRATO             45816 non-null  int64  
 8   TIEMPO ACTIVIDAD          45816 non-null  float64
 9   INGRESOS FIJOS            45816 non-null  int64  
 10  TOTAL INGRESOS            16268 non-null  float64
 11  GASTOS DE SOSTENIMIENTO   45369 non-null  float64
 12  ACTIVOS                   36683 non-null  float64
 13  MONTO PREAPROBADO         45816 non-null  int64  
 14  MONTO 

In [7]:
imputer = SimpleImputer(strategy="median")
X_imp = imputer.fit_transform(X)

Train-Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

Escalamiento

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imp)



In [10]:
X_train.isna().sum()

,0
id,0
NIVEL ESTUDIOS,0
ESTADOCIVIL,0
GÉNERO,853
TIPO VIVIENDA,0
TIEMPO EN RESIDENCIA,0
ESTRATO,0
TIPO CONTRATO,0
TIEMPO ACTIVIDAD,0
INGRESOS FIJOS,0


Lasso con CV

In [11]:
lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_scaled, y)

coef = pd.Series(lasso.coef_, index=X.columns)
variables = coef[coef!=0].sort_values(key=abs, ascending=False)
print(variables)


SALDO VENCIDO        0.154061
MONTO PREAPROBADO   -0.044768
SCORE               -0.040149
id                  -0.038546
PLAZO                0.028888
SALDO CAPITAL        0.027817
NIVEL ESTUDIOS      -0.004079
MONTO DESEMBOLSO    -0.003275
CODIGO CIIU          0.002656
EDAD                 0.001405
GÉNERO              -0.001108
TIPO VIVIENDA       -0.000812
INGRESOS FIJOS      -0.000788
CUOTAMENSUAL        -0.000252
ESTRATO              0.000170
TIEMPO ACTIVIDAD     0.000038
MONTO APROBADO      -0.000033
dtype: float64


XGBoost

In [12]:
top_features = [
    "SCORE",
    "PLAZO",
    "NIVEL ESTUDIOS",
    "EDAD",
    "GÉNERO",
    "CODIGO CIIU",
    "CUOTAMENSUAL",
    "INGRESOS FIJOS",
    "GASTOS DE SOSTENIMIENTO",
    "TIPO VIVIENDA",
    "ESTRATO"
]
X = df[top_features]
y = (df["DIAS DE MORA"] >= 30).astype(int)





In [13]:
X.head()

,SCORE,PLAZO,NIVEL ESTUDIOS,EDAD,GÉNERO,CODIGO CIIU,CUOTAMENSUAL,INGRESOS FIJOS,GASTOS DE SOSTENIMIENTO,TIPO VIVIENDA,ESTRATO
0,649,36,5,42,0.0,8560,632001.190699,13000000,35000.0,2,5
1,843,36,3,60,1.0,4610,232260.179192,19000000,10000.0,1,4
2,630,18,3,20,1.0,4799,91504.941888,1500000,500000.0,2,3
3,754,36,3,29,1.0,9609,115102.274644,4000000,600000.0,1,4
4,503,36,5,18,1.0,4799,84800.457032,16000000,50000.0,1,4


In [14]:
y.value_counts(normalize=True).rename("proporción").round(3)

,proporción
DIAS DE MORA,
0,0.842
1,0.158


In [15]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier



In [16]:
X_train.isna().sum()

,0
id,0
NIVEL ESTUDIOS,0
ESTADOCIVIL,0
GÉNERO,853
TIPO VIVIENDA,0
TIEMPO EN RESIDENCIA,0
ESTRATO,0
TIPO CONTRATO,0
TIEMPO ACTIVIDAD,0
INGRESOS FIJOS,0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

imputer = SimpleImputer(strategy="median")
X_train_imp = imputer.fit_transform(X_train)
X_test_imp  = imputer.transform(X_test)

smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_imp, y_train)

In [18]:
xgb = XGBClassifier(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=3,
    random_state=42,
    n_jobs=-1
)


In [19]:
xgb.fit(X_train_bal, y_train_bal)

y_proba = xgb.predict_proba(X_test_imp)[:, 1]
y_pred_new = (y_proba >= 0.60).astype(int)


In [20]:
print("\n Accuracy:", accuracy_score(y_test, y_pred_new))
print(" Precision :", precision_score(y_test, y_pred_new))
print(" Recall :", recall_score(y_test, y_pred_new))
print(" F1-score mora:", f1_score(y_test, y_pred_new))
print(" ROC-AUC:", roc_auc_score(y_test, y_pred_new))

print("\n Reporte completo:")
print(classification_report(y_test, y_pred_new))

print("\n Matriz de confusión:")
print(confusion_matrix(y_test, y_pred_new))


 Accuracy: 0.8175336485994907
 Precision : 0.31138392857142855
 Recall : 0.12857142857142856
 F1-score mora: 0.18199608610567514
 ROC-AUC: 0.5376334464671397

 Reporte completo:
              precision    recall  f1-score   support

           0       0.85      0.95      0.90     11575
           1       0.31      0.13      0.18      2170

    accuracy                           0.82     13745
   macro avg       0.58      0.54      0.54     13745
weighted avg       0.77      0.82      0.78     13745


 Matriz de confusión:
[[10958   617]
 [ 1891   279]]


In [21]:
def evaluar_threshold(thresholds, y_test, y_proba):
    resultados = []

    for t in thresholds:
        y_pred_t = (y_proba >= t).astype(int)

        resultados.append([
            t,
            precision_score(y_test, y_pred_t),
            recall_score(y_test, y_pred_t),
            f1_score(y_test, y_pred_t)
        ])

    return pd.DataFrame(resultados, columns=["threshold","precision","recall","f1"])

In [22]:
thresholds = np.arange(0.1, 0.91, 0.05)
tabla_thresholds = evaluar_threshold(thresholds, y_test, y_proba)
print(tabla_thresholds)

    threshold  precision    recall        f1
0        0.10   0.168043  0.975576  0.286701
1        0.15   0.178017  0.935945  0.299138
2        0.20   0.189005  0.882488  0.311331
3        0.25   0.199501  0.810138  0.320160
4        0.30   0.213726  0.729032  0.330547
5        0.35   0.225222  0.630415  0.331878
6        0.40   0.240752  0.524885  0.330097
7        0.45   0.255995  0.408295  0.314687
8        0.50   0.276024  0.298157  0.286664
9        0.55   0.292112  0.201382  0.238407
10       0.60   0.311384  0.128571  0.181996
11       0.65   0.347072  0.073733  0.121627
12       0.70   0.366667  0.035484  0.064706
13       0.75   0.397436  0.014286  0.027580
14       0.80   0.375000  0.002765  0.005489
15       0.85   0.500000  0.000461  0.000921
16       0.90   0.000000  0.000000  0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Nuevo treshold

In [23]:
threshold = 0.20
y_pred_new = (y_proba >= threshold).astype(int)

In [24]:
print("\n Accuracy:", accuracy_score(y_test, y_pred_new))
print(" Precision (mora):", precision_score(y_test, y_pred_new))
print(" Recall (mora):", recall_score(y_test, y_pred_new))
print(" F1-score mora:", f1_score(y_test, y_pred_new))

print("\n Matriz de confusión:")
print(confusion_matrix(y_test, y_pred_new))


 Accuracy: 0.38363041105856677
 Precision (mora): 0.18900513225424398
 Recall (mora): 0.8824884792626728
 F1-score mora: 0.31133149081450173

 Matriz de confusión:
[[3358 8217]
 [ 255 1915]]


In [25]:
threshold = 0.30
y_pred_new_2 = (y_proba >= threshold).astype(int)

In [26]:
print("\n Accuracy:", accuracy_score(y_test, y_pred_new_2))
print(" Precision (mora):", precision_score(y_test, y_pred_new_2))
print(" Recall (mora):", recall_score(y_test, y_pred_new_2))
print(" F1-score mora:", f1_score(y_test, y_pred_new_2))

print("\n Matriz de confusión:")
print(confusion_matrix(y_test, y_pred_new_2))


 Accuracy: 0.5337941069479811
 Precision (mora): 0.21372601999459606
 Recall (mora): 0.7290322580645161
 F1-score mora: 0.33054743000417885

 Matriz de confusión:
[[5755 5820]
 [ 588 1582]]


In [27]:
df.count()

,0
id,45816
NIVEL ESTUDIOS,45816
ESTADOCIVIL,45816
GÉNERO,44628
TIPO VIVIENDA,45816
TIEMPO EN RESIDENCIA,45816
ESTRATO,45816
TIPO CONTRATO,45816
TIEMPO ACTIVIDAD,45816
INGRESOS FIJOS,45816
